In [ ]:
# !pip install tensorflow

In [21]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


pd.set_option("display.max_columns",None)

In [22]:
import os
import pandas as pd


main_folder = "/content/drive/MyDrive/data"

all_patients_data = []

for patient_folder in os.listdir(main_folder):
    patient_path = os.path.join(main_folder, patient_folder)

    if os.path.isdir(patient_path):

        blood_file = os.path.join(patient_path, "blood_monitoring.csv")
        bp_file = os.path.join(patient_path, "bp_monitoring.csv")
        lab_file = os.path.join(patient_path, "lab_results.csv")


        blood_df = pd.read_csv(blood_file)
        bp_df = pd.read_csv(bp_file)
        lab_df = pd.read_csv(lab_file)


        merged_df = blood_df.merge(bp_df, on=["Patient_ID", "Date", "Time"], how="outer")


        lab_columns = set(lab_df.columns) - {"Patient_ID", "Date"}


        existing_columns = lab_columns.intersection(merged_df.columns)


        lab_df = lab_df.drop(columns=existing_columns, errors="ignore")


        merged_df = merged_df.merge(lab_df, on=["Patient_ID", "Date"], how="outer")


        merged_df = merged_df.sort_values(by=["Patient_ID", "Date"], ascending=False)


        merged_df[list(existing_columns)] = merged_df[list(existing_columns)].ffill()


        all_patients_data.append(merged_df)
final_df = pd.concat(all_patients_data, ignore_index=True)
final_df=final_df.drop("Lab Result",axis=1)
final_df = final_df.sort_values(by=['Date','Time']).reset_index(drop=True)
final_df = final_df.ffill()
cols = ['Patient_ID'] + [col for col in final_df.columns if col != 'Patient_ID']
final_df = final_df[cols]
final_df=final_df.dropna(axis=0)
final_output_path = "merged_patient_data.csv"
final_df.to_csv(final_output_path, index=False)

print(f"Final merged dataset saved at: {final_output_path}")


Final merged dataset saved at: merged_patient_data.csv


In [18]:
df=pd.read_csv("/content/merged_patient_data.csv")

In [ ]:
df.head()

,Date,Time,Blood Glucose Level (mg/dL),Blood Oxygen (SpO₂),Electrocardiogram (ECG/EKG),Hydration Levels,Patient_ID,Blood Pressure,Heart Rate (HR),Respiratory Rate (RR),Body Temperature,Hemoglobin,Glucose,Cholesterol,Heart Rate,Platelet Count,WBC Count,RBC Count,Creatinine,Urea,Sodium,Potassium,Calcium
0,28-02-2025,0.07.33,125.3,95.4,1.18,67.0,P12346,95/84,62,15,36.9,14.0,144.7,241.4,63.0,195854.0,6611.7,6.26,1.36,35.2,144.2,5.4,10.11
1,28-02-2025,0.17.33,93.9,99.3,1.15,62.9,P12346,95/83,89,13,36.7,14.0,144.7,241.4,63.0,195854.0,6611.7,6.26,1.36,35.2,144.2,5.4,10.11
2,28-02-2025,0.27.33,185.0,97.2,0.89,59.4,P12346,122/67,69,17,37.7,14.0,144.7,241.4,63.0,195854.0,6611.7,6.26,1.36,35.2,144.2,5.4,10.11
3,28-02-2025,0.37.33,108.1,98.1,0.93,70.2,P12346,140/62,76,13,37.9,14.0,144.7,241.4,63.0,195854.0,6611.7,6.26,1.36,35.2,144.2,5.4,10.11
4,28-02-2025,0.47.33,127.6,97.3,0.89,55.9,P12346,135/80,60,19,36.9,14.0,144.7,241.4,63.0,195854.0,6611.7,6.26,1.36,35.2,144.2,5.4,10.11


In [ ]:
df.columns

Index(['Date', 'Time', 'Blood Glucose Level (mg/dL)', 'Blood Oxygen (SpO₂)',
       'Electrocardiogram (ECG/EKG)', 'Hydration Levels', 'Patient_ID',
       'Blood Pressure', 'Heart Rate (HR)', 'Respiratory Rate (RR)',
       'Body Temperature', 'Hemoglobin', 'Glucose', 'Cholesterol',
       'Heart Rate', 'Platelet Count', 'WBC Count', 'RBC Count', 'Creatinine',
       'Urea', 'Sodium', 'Potassium', 'Calcium'],
      dtype='object')

In [19]:
import numpy as np
import pandas as pd
import joblib
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, LeakyReLU, Bidirectional, LayerNormalization, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import AdamW
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# ✅ Convert Date & Time to Timestamp
if 'Date' in df.columns and 'Time' in df.columns:
    df['Timestamp'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format='%d-%m-%Y %H.%M.%S')
    df.drop(columns=['Date', 'Time'], inplace=True)

# 🩸 Split Blood Pressure
if 'Blood Pressure' in df.columns:
    df[['Systolic_BP', 'Diastolic_BP']] = df['Blood Pressure'].str.split('/', expand=True).astype(float)
    df.drop(columns=['Blood Pressure'], inplace=True)

# 🔄 Fill Missing Values

encoder = LabelEncoder()
df['Patient_ID'] = encoder.fit_transform(df['Patient_ID'])
joblib.dump(encoder, "/content/sample_data/bilstm/encoder.pkl")
df = df.drop(columns=['Timestamp'], errors='ignore').copy()


scaler = StandardScaler()

df = df.astype(np.float64)
df_scaled=df.copy()
df_scaled.iloc[:, 1:] = scaler.fit_transform(df_scaled.iloc[:, 1:])
joblib.dump(scaler, "/content/sample_data/bilstm/scaler.pkl")


def create_sequences(data, seq_length=30):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

X, y = create_sequences(df_scaled.iloc[:, 1:].values, seq_length=30)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


model = Sequential([


    Bidirectional(LSTM(128, activation='tanh', return_sequences=True, kernel_regularizer=l2(0.01)), input_shape=(30, X.shape[2])),
    LayerNormalization(),
    Dropout(0.2),

    Bidirectional(LSTM(64, activation='tanh', return_sequences=True)),
    BatchNormalization(),
    Dropout(0.2),

    Bidirectional(LSTM(32, activation='tanh')),
    LayerNormalization(),
    Dropout(0.2),

    Dense(X.shape[2])
])

model.compile(optimizer=AdamW(learning_rate=0.001), loss='mae')


early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

model.fit(X_train, y_train, epochs=20, batch_size=16, validation_data=(X_test, y_test),
          callbacks=[early_stopping, reduce_lr])


y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("\n📊 Model Evaluation:")
print(f"MAE: {mae:.4f}, RMSE: {rmse:.4f}, R² Score: {r2:.4f}")


model.save("/content/sample_data/bilstm/patient_lstm_model.keras")



/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 28s 121ms/step - loss: 1.5447 - val_loss: 0.7939 - learning_rate: 0.0010
Epoch 2/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 14s 111ms/step - loss: 0.8854 - val_loss: 0.6475 - learning_rate: 0.0010
Epoch 3/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 20s 107ms/step - loss: 0.7423 - val_loss: 0.5374 - learning_rate: 0.0010
Epoch 4/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 21s 111ms/step - loss: 0.6398 - val_loss: 0.4871 - learning_rate: 0.0010
Epoch 5/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 20s 111ms/step - loss: 0.5773 - val_loss: 0.4626 - learning_rate: 0.0010
Epoch 6/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 20s 110ms/step - loss: 0.5561 - val_loss: 0.4557 - learning_rate: 0.0010
Epoch 7/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 20s 107ms/step - loss: 0.5227 - val_loss: 0.4419 - learning_rate: 0.0010
Epoch 8/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 14s 106ms/step - loss: 0.5092 - val_loss: 0.4336 - learning_rate: 0.0010
Epoch 9/20
129/129 ━━━━━━━━━━━━━━━━━━━━ 21s 106ms/step - loss: 0.5060 - val_loss: 0.4352 - learn

In [20]:
import joblib

encoder = joblib.load("/content/sample_data/bilstm/encoder.pkl")

scaler = joblib.load("/content/sample_data/bilstm/scaler.pkl")
predictions = {}
for patient_id in df_scaled['Patient_ID'].unique():
    print(patient_id)
    patient_data = df_scaled[df_scaled['Patient_ID'] == patient_id].iloc[:, 1:].values

    if len(patient_data) < 30:
        continue

    last_sequence = patient_data[-30:].reshape(1, 30, -1)
    print(last_sequence)
    next_row_prediction = model.predict(last_sequence)

    if next_row_prediction.shape[1] != scaler.scale_.shape[0]:
        continue

    next_row_original = scaler.inverse_transform(next_row_prediction)
    predictions[patient_id] = np.round(next_row_original, 2)

for patient_id, pred in predictions.items():

    decoded_patient_id = encoder.inverse_transform([int(patient_id)])[0]

    print(f"\n🔮 Predicted Next Readings for Patient {decoded_patient_id}:")
    print(pred)


2.0
[[[ 0.69904988  1.70665477 -1.22375879  1.5832376   0.93318511
   -0.68659272 -1.8924335  -0.04813652 -0.42547387  1.25132585
   -1.74364809 -0.52918415 -0.35638827  2.14264701  1.814715
    1.0795378   1.4410725   1.18198373  0.71840964  0.67435273
   -0.07786177]
  [-1.48207424 -1.63059191 -1.31385431  1.06503479  1.10256404
    0.56286784  0.77549805 -0.04813652 -0.42547387  1.25132585
   -1.74364809 -0.52918415 -0.35638827  2.14264701  1.814715
    1.0795378   1.4410725   1.18198373  0.71840964  1.03531471
   -0.53113003]
  [ 1.25219553 -1.88245958  1.20882021  1.80532452  0.59442725
   -0.68659272 -0.55846773 -0.04813652 -0.42547387  1.25132585
   -1.74364809 -0.52918415 -0.35638827  2.14264701  1.814715
    1.0795378   1.4410725   1.18198373  0.71840964  1.54066148
    1.62189423]
  [ 1.3832727  -1.81949267 -0.95347223  0.33955084 -0.84529365
    0.9793547   0.20379843 -0.04813652 -0.42547387  1.25132585
   -1.74364809 -0.52918415 -0.35638827  2.14264701  1.814715
    1.07953

In [ ]:
!pip cache purge  # Clears cached packages
!pip install --no-cache-dir pytorch-lightning pytorch-forecasting torchmetrics


Files removed: 0


In [11]:


import torch
from transformers import TimeSeriesTransformerForPrediction, TimeSeriesTransformerConfig
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Load the dataset
file_path = "bp_monitoring_with_report(1).csv"
df = pd.read_csv(file_path)

# Convert Date and Time to a single Datetime column
df["Datetime"] = pd.to_datetime(df["Date"] + " " + df["Time"], format="%d-%m-%Y %H.%M.%S")

# Split Blood Pressure into Systolic and Diastolic
df[["Systolic", "Diastolic"]] = df["Blood Pressure"].str.split("/", expand=True).astype(int)

# Drop unnecessary columns
df = df.drop(columns=["Date", "Time", "Blood Pressure", "Report"])

# Sort by Datetime
df = df.sort_values("Datetime").reset_index(drop=True)

# Select numerical columns
numerical_cols = ["Systolic", "Diastolic", "Heart Rate (HR)", "Respiratory Rate (RR)", "Body Temperature"]

# Normalize data
scaler = MinMaxScaler()
df_scaled = df.copy()
df_scaled[numerical_cols] = scaler.fit_transform(df[numerical_cols])

# Convert data to numpy array
data_values = df_scaled[numerical_cols].values

# Prepare sequences
sequence_length = 10
X, y = [], []

for i in range(len(data_values) - sequence_length):
    X.append(data_values[i:i + sequence_length])
    y.append(data_values[i + sequence_length])

X, y = np.array(X), np.array(y)

# Convert data to tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

# Load Pretrained Hugging Face Model
model_name = "facebook/time-series-transformer"  # Example pretrained model
model = TimeSeriesTransformerForPrediction.from_pretrained(model_name)

# Make Predictions
with torch.no_grad():
    next_input = X_tensor[-1].unsqueeze(0)  # Add batch dimension
    predictions = model(next_input).logits

# Convert back to original scale
next_prediction_actual = scaler.inverse_transform(predictions.numpy())

# Print Predicted Next Observation
print("Next Predicted Observation:", next_prediction_actual)


Exception ignored in: <function _xla_gc_callback at 0x7f75a7d39da0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/jax/_src/lib/__init__.py", line 96, in _xla_gc_callback
    def _xla_gc_callback(*args):
    
KeyboardInterrupt: 
Exception ignored in: <function _xla_gc_callback at 0x7f75a7d39da0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/jax/_src/lib/__init__.py", line 96, in _xla_gc_callback
    def _xla_gc_callback(*args):
    
KeyboardInterrupt: 


KeyboardInterrupt: 